<a href="https://colab.research.google.com/github/Forbusinessuseyukikoishiguro/-/blob/main/%E9%8C%B2%E9%9F%B3%E3%83%87%E3%83%A2%E3%83%86%E3%82%B9%E3%83%88OK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from IPython.display import display, HTML
import base64
import io
import os

# 録音用のダイアログを表示する関数
def record_audio():
    display(HTML('''
        <button id="startBtn" onclick="startRecording()">録音開始</button>
        <button id="stopBtn" onclick="stopRecording()" disabled>録音停止</button>
        <audio id="audio" controls></audio>
        <script>
            let mediaRecorder;
            let audioChunks = [];
            let audioElement = document.getElementById('audio');
            let startBtn = document.getElementById('startBtn');
            let stopBtn = document.getElementById('stopBtn');

            async function startRecording() {
                const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
                mediaRecorder = new MediaRecorder(stream);
                mediaRecorder.start();
                mediaRecorder.ondataavailable = event => {
                    audioChunks.push(event.data);
                };
                startBtn.disabled = true;
                stopBtn.disabled = false;
            }

            async function stopRecording() {
                mediaRecorder.stop();
                mediaRecorder.onstop = async () => {
                    const audioBlob = new Blob(audioChunks, { type: 'audio/wav' });
                    const reader = new FileReader();
                    reader.readAsDataURL(audioBlob);  // Base64形式に変換
                    reader.onloadend = function() {
                        const base64data = reader.result.split(',')[1];  // Base64データ部分のみを取得
                        google.colab.kernel.invokeFunction('recordedAudio', [base64data], {});
                        audioElement.src = URL.createObjectURL(audioBlob);  // 録音した音声を再生
                    };
                    audioChunks = [];
                };
                startBtn.disabled = false;
                stopBtn.disabled = true;
            }
        </script>
    '''))

# Python側で受け取る関数を定義
from google.colab import output

# 音声データを格納する変数
audio_data = None

def handle_recorded_audio(base64_audio):
    global audio_data
    audio_data = base64_audio  # 録音された音声をBase64形式で格納

    # Base64形式のデータをデコードしてファイルに保存
    audio_bytes = base64.b64decode(base64_audio)
    with open('tmp.wav', 'wb') as f:
        f.write(audio_bytes)

    # FFmpegで音声を変換
    os.system("ffmpeg -y -i tmp.wav -acodec pcm_s16le -ar 44100 recorded_audio.wav")

output.register_callback('recordedAudio', handle_recorded_audio)

# 音声録音を開始
record_audio()